In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.legacy import Adam, SGD
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, matthews_corrcoef, cohen_kappa_score
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import numpy as np

import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

def learning_schedule(epoch):
    if epoch < 10:
        return 0.001
    elif epoch < 20:
        return 0.0001
    else:
        return 0.00001

lr_schedule = LearningRateScheduler(learning_schedule)

class BrainTumorDetector:
    def __init__(self, dataset_dir, img_width=224, img_height=224, batch_size=16):
        self.dataset_dir = dataset_dir
        self.img_width = img_width
        self.img_height = img_height

        self.batch_size = batch_size
        self.binary_classifier = None
        self.multiclass_classifier = None
        self.binary_classification_report = None
        self.multiclass_classification_report = None

    def data_process(self, stage, subset):
        if stage == "binary_classification":
            data_dir = f"{self.dataset_dir}/{subset}/binary_classification"
        elif stage == "multiclass_classification":
            data_dir = f"{self.dataset_dir}/{subset}/multiclass_classification"
        else:
            raise ValueError("Invalid stage. Use 'binary_classification' or 'multiclass_classification'.")

        datagen = ImageDataGenerator(
            rescale=1.0 / 255,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )

        generator = datagen.flow_from_directory(
            data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=self.batch_size,
            class_mode='binary' if stage == "binary_classification" else 'categorical'
        )

        return generator

    def build_binary_classifier(self):
        base_model = VGG16(weights='/content/drive/MyDrive/Colab Notebooks/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(self.img_width, self.img_height, 3))

        x = base_model.output
        x = GlobalAveragePooling2D()(x)

        x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
        x = Dropout(0.2)(x)

        x = Dense(256, activation='relu')(x)




        output = Dense(1, activation='sigmoid')(x)

        self.binary_classifier = Model(inputs=base_model.input, outputs=output)

        for layer in base_model.layers:
            layer.trainable = False

        self.binary_classifier.compile(optimizer=SGD(learning_rate=0.001, clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])

    def train_binary_classifier(self, generator, epochs=100):
        unique_classes = list(set(generator.classes))
        class_weights = compute_class_weight('balanced', classes=unique_classes, y=generator.classes)
        steps_per_epoch = generator.samples // self.batch_size

        self.binary_classifier.fit(
            generator,
            steps_per_epoch=steps_per_epoch,
            epochs=epochs,
            class_weight=dict(enumerate(class_weights)),
            callbacks=[lr_schedule],
            verbose=1
        )

    def evaluate_binary_classifier(self, generator):
        test_loss, test_acc = self.binary_classifier.evaluate(generator, verbose=0)
        print(f"Binary Classification - Test Loss: {test_loss:.4f}")
        print(f"Binary Classification - Test Accuracy: {test_acc * 100:.2f}%")

        y_true = generator.classes
        y_pred = self.binary_classifier.predict(generator)

        self.binary_classification_report = classification_report(y_true, y_pred.argmax(axis=-1), target_names=['No Tumor', 'Tumor'], zero_division=1)

        # Calculate AUC-ROC
        y_prob = self.binary_classifier.predict(generator)
        auc_roc = roc_auc_score(y_true, y_prob)
        print(f"AUC-ROC: {auc_roc:.4f}")

        # Calculate Matthews Correlation Coefficient (MCC)
        mcc = matthews_corrcoef(y_true, y_pred.argmax(axis=-1))
        print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

        # Calculate Cohen's Kappa
        kappa = cohen_kappa_score(y_true, y_pred.argmax(axis=-1))
        print(f"Cohen's Kappa: {kappa:.4f}")

    def build_multiclass_classifier(self):
        base_model = VGG16(weights='/content/drive/MyDrive/Colab Notebooks/weights.h5', include_top=False, input_shape=(self.img_width, self.img_height, 3))

        x = base_model.output
        x = GlobalAveragePooling2D()(x)

        x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
        x = Dropout(0.5)(x)

        x = Dense(256, activation='relu')(x)

        x = Dense(128, activation='relu')(x)



        output = Dense(3, activation='softmax', kernel_regularizer=l2(0.01))(x)

        self.multiclass_classifier = Model(inputs=base_model.input, outputs=output)

        for layer in base_model.layers:
            layer.trainable = False

        self.multiclass_classifier.compile(optimizer=SGD(learning_rate=0.001, clipnorm=1.0), loss='categorical_crossentropy', metrics=['accuracy'])

    def train_multiclass_classifier(self, generator, epochs=100):
        unique_classes = list(set(generator.classes))
        class_weights = compute_class_weight('balanced', classes=unique_classes, y=generator.classes)
        steps_per_epoch = generator.samples // self.batch_size

        self.multiclass_classifier.fit(
            generator,
            steps_per_epoch=steps_per_epoch,
            epochs=epochs,
            class_weight=dict(enumerate(class_weights)),
            callbacks=[lr_schedule],
            verbose=1
        )

    def evaluate_multiclass_classifier(self, generator):
        test_loss, test_acc = self.multiclass_classifier.evaluate(generator, verbose=0)
        print(f"Multiclass Classification - Test Loss: {test_loss:.4f}")
        print(f"Multiclass Classification - Test Accuracy: {test_acc * 100:.2f}%")

        y_true = generator.classes
        y_pred = self.multiclass_classifier.predict(generator)
        # Generate the classification report
        self.multiclass_classification_report = classification_report(y_true, y_pred.argmax(axis=-1), target_names=['Glioma Tumor', 'Meningioma Tumor', 'Pituitary Tumor'], zero_division=1, output_dict=True)

        # Extract and print accuracy by class
        accuracy_by_class = {label: self.multiclass_classification_report[label]['precision'] for label in self.multiclass_classification_report if label in ['Glioma Tumor', 'Meningioma Tumor', 'Pituitary Tumor']}
        for label, accuracy in accuracy_by_class.items():
            print(f"{label} - Accuracy: {accuracy * 100:.2f}%")

        # Calculate Matthews Correlation Coefficient (MCC)
        mcc = matthews_corrcoef(y_true, y_pred.argmax(axis=-1))
        print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

        # Calculate Cohen's Kappa
        kappa = cohen_kappa_score(y_true, y_pred.argmax(axis=-1))
        print(f"Cohen's Kappa: {kappa:.4f}")



    def generate_total_classification_report(self):
        if self.binary_classification_report is not None and self.multiclass_classification_report is not None:
            self.total_classification_report = f"Binary Classification Report:\n{self.binary_classification_report}\n\nMulticlass Classification Report:\n{self.multiclass_classification_report}"
        else:
            self.total_classification_report = "Classification reports not available."


    def print_total_classification_report(self):
        print(self.total_classification_report)
if __name__ == '__main__':
    dataset_dir = '/content/drive/MyDrive/Colab Notebooks/dataset'  # Adjust the path to your dataset folder
    batch_size = 1

    brain_tumor_detector = BrainTumorDetector(dataset_dir)

    # Stage 1: Binary Classification (No Tumor vs. Tumor) - Training
    binary_classification_generator_train = brain_tumor_detector.data_process("binary_classification", "Training")
    brain_tumor_detector.build_binary_classifier()
    brain_tumor_detector.train_binary_classifier(binary_classification_generator_train, epochs=1)

    # Stage 1: Binary Classification - Testing
    binary_classification_generator_test = brain_tumor_detector.data_process("binary_classification", "Testing")
    brain_tumor_detector.evaluate_binary_classifier(binary_classification_generator_test)

    # Stage 2: Multiclass Classification (Tumor Type) - Training
    multiclass_classification_generator_train = brain_tumor_detector.data_process("multiclass_classification", "Training")
    brain_tumor_detector.build_multiclass_classifier()
    brain_tumor_detector.train_multiclass_classifier(multiclass_classification_generator_train, epochs=1)

    # Stage 2: Multiclass Classification - Testing
    multiclass_classification_generator_test = brain_tumor_detector.data_process("multiclass_classification", "Testing")
    brain_tumor_detector.evaluate_multiclass_classifier(multiclass_classification_generator_test)

    # Store binary and multiclass classification reports separately
    binary_classification_report = brain_tumor_detector.binary_classification_report
    multiclass_classification_report = brain_tumor_detector.multiclass_classification_report

    # Generate and print the total classification report
    brain_tumor_detector.generate_total_classification_report()
    brain_tumor_detector.print_total_classification_report()

    # Print binary and multiclass classification reports separately
    print("\nBinary Classification Report:")
    print(binary_classification_report)

    print("\nMulticlass Classification Report:")
    print(multiclass_classification_report)

Found 2870 images belonging to 2 classes.
179/179 [==============================] - 63s 270ms/step - loss: 1.2080 - accuracy: 0.7754 - lr: 0.0010
Found 394 images belonging to 2 classes.
Binary Classification - Test Loss: 1.1980
Binary Classification - Test Accuracy: 58.88%
25/25 [==============================] - 6s 238ms/step
AUC-ROC: 0.4885
Matthews Correlation Coefficient (MCC): 0.0000
Cohen's Kappa: 0.0000
Found 2475 images belonging to 3 classes.
154/154 [==============================] - 631s 4s/step - loss: 1.6840 - accuracy: 0.3587 - lr: 0.0010
Found 289 images belonging to 3 classes.
Multiclass Classification - Test Loss: 1.6540
Multiclass Classification - Test Accuracy: 38.41%
19/19 [==============================] - 5s 237ms/step
Glioma Tumor - Accuracy: 36.33%
Meningioma Tumor - Accuracy: 0.00%
Pituitary Tumor - Accuracy: 27.91%
Matthews Correlation Coefficient (MCC): 0.0404
Cohen's Kappa: 0.0250
Binary Classification Report:
              precision    recall  f1-score   